In [14]:
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder \
    .appName("HELK JOIN") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [16]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [17]:
security_df = es_reader.load("logs-endpoint-winevent-security-*/")

In [18]:
security_df.createOrReplaceTempView("security_events")

In [38]:
security_4769 = spark.sql(
'''
    SELECT event_id, user_name, ticket_encryption_type_value, service_ticket_name
    FROM security_events 
    WHERE event_id = 4769
        AND ticket_encryption_type_value = "RC4-HMAC"
        AND NOT user_name LIKE "%$"
        AND NOT(
        service_ticket_name LIKE "%$" AND service_ticket_name = "krbtgt"
        )
'''
)
security_4769.show()

+--------+-----------------+----------------------------+-------------------+
|event_id|        user_name|ticket_encryption_type_value|service_ticket_name|
+--------+-----------------+----------------------------+-------------------+
|    4769|nmartha@shire.com|                    RC4-HMAC|            gandalf|
+--------+-----------------+----------------------------+-------------------+

